### Dependencies
pymesh: http://pymesh.readthedocs.io/en/latest/

pypcl: https://github.com/cmpute/pypcl

sklearn

pandas

numpy

In [1]:
import pymesh
import os
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
#import pcl

In [2]:
MESHES_DIR = "PlantDataSurface"
PCL_DIR = "PointCloudsLabels"
OUTPUT_DIR = "Output"

USE_BINARY_PCL = False
GENERATE_OBJS = True
GENERATE_LABELS = True

STEM = 0
LEAF = 1

In [3]:
def process_directory(directory, function, file_ext=None):
    for entry in os.listdir(directory):
        path = os.path.join(directory, entry)
        if os.path.isdir(path):
            process_directory(path, function, file_ext)
        elif path.lower().endswith(file_ext.lower()):
            function(path)

In [4]:
def get_file_name(filepath):
    return os.path.basename(filepath).split(".")[0]

In [5]:
pcl_files = {}
def registerPointCloudFile(pcl_file):
    global pcl_files
    pcl_name = get_file_name(pcl_file).lower()
    if pcl_name.startswith("ctrl"):
        pcl_name = "control" + pcl_name[4:]
    pcl_files[pcl_name] = pcl_file

In [6]:
if USE_BINARY_PCL:
    process_directory(PCL_DIR, registerPointCloudFile, ".pcd")
else:
    process_directory(PCL_DIR, registerPointCloudFile, ".txt")

In [ ]:
def generate_labeled_meshes(mesh_file):
    file_name = get_file_name(mesh_file)
    pcl_stem_file = file_name.lower() + "_s"
    pcl_leaf_file = file_name.lower() + "_l"
    if pcl_stem_file not in pcl_files or \
       pcl_leaf_file not in pcl_files:
        print("No point cloud file found for mesh", mesh_file)
        return
    
    print("Processing ", mesh_file, "...", sep="")
    
    # Load mesh and its labeled point cloud
    mesh = pymesh.load_mesh(mesh_file)
    
    if USE_BINARY_PCL:
        pcl_stem = pcl.io.PCDReader().read(pcl_files[pcl_stem_file])
        pcl_leaf = pcl.io.PCDReader().read(pcl_files[pcl_leaf_file])
    else:
        pcl_stem = pd.read_csv(pcl_files[pcl_stem_file], sep=",", names=["x", "y", "z"]).as_matrix()
        pcl_leaf = pd.read_csv(pcl_files[pcl_leaf_file], sep=",", names=["x", "y", "z"]).as_matrix()
    
    
    # Create an array which gives the label of every vertex
    # in the combined stem and leaf point clouds
    labels = np.concatenate((np.full((pcl_stem.shape[0]), STEM),
                             np.full((pcl_leaf.shape[0]), LEAF)))

    # Concatenate the stem and leaf point clouds
    points = np.concatenate((pcl_stem, pcl_leaf))
    
    # For every vertex, find the k nearest points in the point cloud
    k = 3
    knn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(points)
    distances, indices = knn.kneighbors(mesh.vertices)

    # Create per-face labels
    per_face_label = np.zeros(len(mesh.faces), dtype=np.int32)
    for i in range(len(mesh.faces)):
        votes = [0, 0]
        # For every vertex in this triangle, count the labels
        # of the k closest points in the point cloud
        for label in labels[indices[mesh.faces[i]].flatten()]:
            votes[label] += 1
        per_face_label[i] = np.argmax(votes)
    
    if GENERATE_OBJS:
        # Generate one mesh for each label
        vertices = [[] for _ in range(2)]
        faces    = [[] for _ in range(2)]
        for i in range(len(mesh.faces)):
            label = per_face_label[i]
            offset = len(vertices[label])
            vertices[label].extend(mesh.vertices[mesh.faces[i]])
            faces[label].append([offset, offset+1, offset+2])

        # Save meshes
        output_dir = OUTPUT_DIR + mesh_file[mesh_file.find(os.sep):mesh_file.rfind(os.sep)]
        if not os.path.isdir(output_dir): os.makedirs(output_dir)
        
        pymesh.save_mesh_raw(os.path.join(output_dir, file_name + "_s.obj"), np.array(vertices[0]), np.array(faces[0]), ascii=True)
        print("Object file created ", os.path.join(output_dir, file_name + "_l.obj"), "...", sep="")
              
        pymesh.save_mesh_raw(os.path.join(output_dir, file_name + "_l.obj"), np.array(vertices[1]), np.array(faces[1]), ascii=True)
        print("Object file created ", os.path.join(output_dir, file_name + "_s.obj"), "...", sep="")
        
    if GENERATE_LABELS:
        # Save per-face labels to file
        output_dir = OUTPUT_DIR + mesh_file[mesh_file.find(os.sep):mesh_file.rfind(os.sep)]
        if not os.path.isdir(output_dir): os.makedirs(output_dir)
        with open(os.path.join(output_dir, file_name + "_labels.txt"), "w") as f:
            f.write("Plant_labelA\n")
            for i in range(len(mesh.faces)):
                if per_face_label[i] == STEM:
                    f.write(str(i) + " ")
            f.write("\n\nPlant_labelB\n")
            for i in range(len(mesh.faces)):
                if per_face_label[i] == LEAF:
                    f.write(str(i) + " ")
            f.write("\n")
        print("Label file created ", os.path.join(output_dir, file_name + "_labels.txt"), "...", sep="")
            
process_directory(MESHES_DIR, generate_labeled_meshes, ".obj")